In [1]:
import os
os.chdir("../")

In [2]:
import os
import json
from dataclasses import dataclass
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from src.constants import *
from src.utils.utils import *
from src.logging import logging

In [3]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    data_file: Path
    train_dir: Path
    test_dir: Path
    model_dir: Path
    metrics_file: Path

In [4]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            data_file=Path(config.data_file),
            train_dir=Path(config.train_dir),
            test_dir=Path(config.test_dir),
            model_dir=Path(config.model_dir),
            metrics_file=Path(config.metrics_file)
        )

        return model_evaluation_config

In [5]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig, params_path=PARAMS_FILE_PATH):
        self.config = config
        self.params = read_yaml(params_path).model_trainer

    def evaluate_model(self):
        try:
            logging.info("Starting model evaluation..")
            test_df = pd.read_csv(self.config.test_dir)
            X_test, y_test = test_df.drop("y", axis=1), test_df["y"]
            model = load_pickle(self.config.model_dir)
            if model:
                y_pred = model.predict(X_test)
            else:
                raise ValueError("Model File Not Found")

            accuracy = accuracy_score(y_test, y_pred)
            f1score = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            metrics = {
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "f1score": f1score,
            }

            save_json(self.config.metrics_file, metrics)
            logging.info("Model Evaluation Completed")
            
            return metrics
        except Exception as e:
            raise e


In [6]:
config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()
model_evaluation = ModelEvaluation(model_evaluation_config)
model_evaluation.evaluate_model()

[2025-08-16 21:49:42,120: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2025-08-16 21:49:42,123: INFO: utils: yaml file: params.yaml loaded successfully]
[2025-08-16 21:49:42,124: INFO: utils: Created Directory at: artifacts]
[2025-08-16 21:49:42,126: INFO: utils: Created Directory at: artifacts/model_evaluation]
[2025-08-16 21:49:42,128: INFO: utils: yaml file: params.yaml loaded successfully]
[2025-08-16 21:49:42,129: INFO: 1869341900: Starting model evaluation..]
[2025-08-16 21:49:42,863: INFO: 1869341900: Model Evaluation Completed]


{'accuracy': 0.8914077186774301,
 'precision': 0.5416348357524828,
 'recall': 0.6498625114573785,
 'f1score': 0.5908333333333333}